<a href="https://colab.research.google.com/github/RoboTuan/FPAR/blob/master/save_Cams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn
import tensorflow as tf
import torchvision
import numpy as np
from torchvision import transforms
import cv2

from PIL import Image


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip -q "./drive/My Drive/GTEA61.zip"
!rm -rf "./__MACOSX"

In [4]:
!rm -rf "./ML_DL_Project"

In [5]:
import os 
if not os.path.isdir('./ML_DL_Project'):
  !git clone https://github.com/RoboTuan/ML_DL_Project.git

Cloning into 'ML_DL_Project'...
remote: Enumerating objects: 263, done.
remote: Counting objects: 100% (263/263), done.
remote: Compressing objects: 100% (252/252), done.
remote: Total 793 (delta 179), reused 25 (delta 11), pack-reused 530
Receiving objects: 100% (793/793), 92.95 MiB | 22.12 MiB/s, done.
Resolving deltas: 100% (534/534), done.


In [ ]:
from ML_DL_Project.Scripts.objectAttentionModelConvLSTM import *

from ML_DL_Project.Scripts.attentionMapModel import attentionMap

In [6]:
base_dataset = "./GTEA61"

if not os.path.isdir('./GTEA61_val'):
  os.makedirs("./GTEA61_val")
  os.makedirs("./GTEA61_val/flow_x_processed")
  os.makedirs("./GTEA61_val/flow_y_processed")
  os.makedirs("./GTEA61_val/processed_frames2")
  pass


for directory in sorted(os.listdir(base_dataset)):
  if not directory.startswith('.'):
    directory_path = os.path.join(base_dataset, directory)
    new_directory_path = os.path.join("./GTEA61_val", directory)

    for user in sorted(os.listdir(directory_path)):
      if not user.startswith('.') and user == "S2":
        current_folder = os.path.join(directory_path, user)
        !cp -r "{current_folder}" "{new_directory_path}"
        !rm -rf "{current_folder}"
        print(current_folder)


./GTEA61/flow_x_processed/S2
./GTEA61/flow_y_processed/S2
./GTEA61/processed_frames2/S2


In [72]:
!rm -rf 'CAMs'

In [49]:
  if not os.path.isdir('./CAMs'):
    os.makedirs('./CAMs')

In [71]:

def takeError(examinedClass,model_path,model_type,selfsup_bool,regression_bool):
  stackSize=1
  Dataset = []
  Labels = []
  NumFrames = []
  directory = examinedClass
  # print for debugging
  print(f"directory: {directory}")
  action = directory.split('/')[-1]
  if not os.path.isdir(f'./CAMs/{model_type}'):
    os.makedirs(f'./CAMs/{model_type}')
    os.makedirs(f'./CAMs/{model_type}/' + action)
  elif not os.path.isdir(f'./CAMs/{model_type}/'+ action):
    os.makedirs(f'./CAMs/{model_type}/' + action)

  
  for target in sorted(os.listdir(directory)):
    #print(f"target: {target}")
    if not target.startswith('.'):
      #print(target)
      directory1 = os.path.join(directory, target)
      # print for debugging
      #print(f"directory1: {directory1}")
      insts = sorted(os.listdir(directory1))
      # print for debugging
      #print(f"insts: {insts}")
      #insts = insts + "/rgb"
      inst_dir= os.path.join(directory1,insts[1])
      #print(f"inst_dir {inst_dir}")


      for image in sorted(os.listdir(inst_dir)):
        image_path= os.path.join(inst_dir, image)
        #print(f"image_path :{image_path}")

        ###################################################################
        num_classes = 61 # Classes in the pre-trained model
        mem_size = 512
        model_state_dict = model_path # Weights of the pre-trained model

        if (selfsup_bool == True):
          model = SelfSupAttentionModel(num_classes=num_classes, mem_size=mem_size, REGRESSOR=regression_bool)
        else:
          model = attentionModel(num_classes=num_classes, mem_size=mem_size)
        model.load_state_dict(torch.load(model_state_dict))
        model_backbone = model.resNet
        attentionMapModel = attentionMap(model_backbone).cuda()
        attentionMapModel.train(False)
        for params in attentionMapModel.parameters():
            params.requires_grad = False
        ###################################################################

        normalize = transforms.Normalize(
          mean=[0.485, 0.456, 0.406],
          std=[0.229, 0.224, 0.225]
        )
        preprocess1 = transforms.Compose([
          transforms.Scale(256),
          transforms.CenterCrop(224),
        ])

        preprocess2 = transforms.Compose([
            transforms.ToTensor(),
            normalize])


        fl_name_in = image_path
        #print(f"fl_name_in :{fl_name_in}")
        fl_name_out = "./CAMs/{}/{}/{}_{}.jpg".format(model_type,action, str(target),fl_name_in.split('/')[-1].split('.')[0])
        #print(f"fl_name_out :{fl_name_out}")
        img_pil = Image.open(fl_name_in)
        img_pil1 = preprocess1(img_pil)
        img_size = img_pil1.size
        size_upsample = (img_size[0], img_size[1])
        img_tensor = preprocess2(img_pil1)
        img_variable = Variable(img_tensor.unsqueeze(0).cuda())
        img = np.asarray(img_pil1)
        attentionMap_image = attentionMapModel(img_variable, img, size_upsample)
        cv2.imwrite(fl_name_out, attentionMap_image)



## RGB MODEL

In [53]:

!unzip -q "./drive/MyDrive/Runs/Parte 1/7FramesRGB_Attention.zip"
!rm -rf "./__MACOSX"

unzip:  cannot find or open ./drive/MyDrive/Runs/Parte 1/7FramesRGB_Attention.zip, ./drive/MyDrive/Runs/Parte 1/7FramesRGB_Attention.zip.zip or ./drive/MyDrive/Runs/Parte 1/7FramesRGB_Attention.zip.ZIP.


In [54]:
!rm -rf "./CAMs/RGB_7"

In [73]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./7FramesRGB_Attention/model_rgb_state_dict(1).pth',model_type='RGB_7',
          selfsup_bool=False,regression_bool=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./7FramesRGB_Attention/model_rgb_state_dict(1).pth',model_type='RGB_7',
          selfsup_bool=False,regression_bool=False) #save cams of the class that is always selected instead of the previous one

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


##RGB MODEL 16 FRAMES

In [41]:
!unzip -q "./drive/MyDrive/Runs/Parte 1/16FramesRGB_Attention.zip" 
!rm -rf "./__MACOSX"

replace 16FramesRGB_Attention/close_honey_rgb_attention1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [74]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='RGB_16',
          selfsup_bool=False,regression_bool=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='RGB_16',
          selfsup_bool=False,regression_bool=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


## SELF SUP MODEL

In [26]:
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *

In [30]:
!unzip -q "./drive/MyDrive/Runs/Parte 2/No Regression/Decay rate optimization/cartella_selfSupDecayRate0,5_prova2.zip"
!rm -rf "./__MACOSX"

replace outDir/gtea61/selfSup/stage2/events.out.tfevents.1608580867.a8826bb2cf35.57.2? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


In [75]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./outDir/gtea61/selfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup',
          selfsup_bool=True,regression_bool=False) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./outDir/gtea61/selfSup/stage2/model_selfSup_state_dict.pth',model_type='self_sup',
          selfsup_bool=True,regression_bool=False) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/pour_chocolate,bread


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/open_chocolate


## SELF SUP REGRESSION MODEL

In [33]:
from ML_DL_Project.Scripts.SelfSupObjectAttentionModelConvLSTM import *

In [34]:
!unzip -q "./drive/MyDrive/Runs/Parte 2/Regression/Decay rate optimization/cartella_RegselfSupGirodecayRate0.3_prova1.zip" 
!rm -rf "./__MACOSX"

replace cartella_RegselfSupGirodecayRate0.3_prova1/mmap_train_loss.svg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [36]:
takeError("./GTEA61_val/processed_frames2/S2/pour_chocolate,bread",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='self_sup_regr',
          selfsup_bool=True,regression_bool=True) #save cams of one of the never predicted class
takeError("./GTEA61_val/processed_frames2/S2/open_chocolate",model_path='./16FramesRGB_Attention/model_rgb_state_dict (1).pth',model_type='self_sup_regr',
          selfsup_bool=True,regression_bool=True) #save cams of the class that is always selected instead of the previous one-

directory: ./GTEA61_val/processed_frames2/S2/scoop_coffee,spoon


/usr/local/lib/python3.6/dist-packages/torchvision/transforms/transforms.py:280: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  "please use transforms.Resize instead.")


directory: ./GTEA61_val/processed_frames2/S2/take_coffee
